In [1]:
import pandas as pd 
import json

In [2]:
df  = pd.read_pickle('job_20k_clean.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19906 entries, 3119 to 3632
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   clean_descrition   19906 non-null  object
 1   organization_list  19906 non-null  object
 2   skill_list         19906 non-null  object
 3   job_title_list     19906 non-null  object
dtypes: object(4)
memory usage: 777.6+ KB


In [3]:
jobs = list(df.clean_descrition)

In [4]:
def convert_to_list_then_dict(df_column= None, key_value=str):
    value_list = list(df_column)
    _ = list(set([item for value_list in value_list for item in value_list]))
    return dict.fromkeys(_, key_value)

In [5]:
org = convert_to_list_then_dict(df_column=df.organization_list, key_value='C')
skill = convert_to_list_then_dict(df_column=df.skill_list, key_value='S')
job = convert_to_list_then_dict(df_column=df.job_title_list, key_value='J')

In [6]:
# def fixed_training_sample(num_samples=int):
#     num_samples = num_samples
    
#     jobs = list(df.clean_descrition[:num_samples])
#     org = convert_to_list_then_dict(df_column=df.organization_list, key_value='C')
#     skill = convert_to_list_then_dict(df_column=df.skill_list, key_value='S')
#     job = convert_to_list_then_dict(df_column=df.job_title_list, key_value='J')
    
#     return jobs,org,skill,job

# jobs,org,skill,job = fixed_training_sample(num_samples=200)

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [8]:
pip install -U 'scikit-learn<0.24'

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from nltk.tokenize import word_tokenize

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [10]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1131)>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify
[nltk_data]     failed: unable to get local issuer certificate
[nltk_data]     (_ssl.c:1131)>


False

In [11]:
# split
XTr= jobs[:1500]
XVal= jobs[1500:2000]
org_b = org
skills_b = skill
jobs_b = job

In [12]:
# XTr=['Google wants a backend developer with good coding skills','There is a position open for machine learning engineer, Meta is looking for people with machine learning background']
# XVal=['Amazon is looking for DevOps engineer']
# org_b={'google':'C','amazon':'C','meta':'C','microsoft':'C','qualcomm':'C'}
# skills_b={'coding':'S','machine':'S','learning':'S'}
# jobs_b={'backend':'J','developer':'J','engineer':'J','manager':'J','devops':'J'}
def make(X,org_b,skills_b,jobs_b):
  big=[]
  for sentence in X:
    token=word_tokenize(sentence)
    small=[]
    token_t=nltk.pos_tag(token)
    for f,s in token_t:
      tag='O'
      if f.lower() in org_b:
        tag=org_b[f.lower()]
      if f.lower() in skills_b:
        tag=skills_b[f.lower()]
      if f.lower() in jobs_b:
        tag=jobs_b[f.lower()]
      small.append((f,s,tag))
    big.append(small)
  return big
XTr_=make(XTr,org_b,skills_b,jobs_b)
XVal_=make(XVal,org_b,skills_b,jobs_b)

In [13]:
def getfeatures(dat,i):
  word=dat[i][0]
  postag=dat[i][1]
  features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
  }
  if i>0:
    word=dat[i-1][0]
    postag=dat[i-1][1]
    features.update({
        '-1:word.lower()': word.lower(),    
        '-1:word.isupper()': word.isupper(),
        '-1:word.istitle()': word.istitle(),
        '-1:postag': postag,
        '-1:postag[:2]': postag[:2]
    })
  else:
    features['BOS']=True
  if i+1<len(dat):
    word=dat[i+1][0]
    postag=dat[i+1][1]
    features.update({
        '+1:word.lower()': word.lower(),    
        '+1:word.isupper()': word.isupper(),
        '+1:word.istitle()': word.istitle(),
        '+1:postag': postag,
        '+1:postag[:2]': postag[:2]
    })
  else:
    features['EOS']=True
  return features

In [14]:
def f2features(dat):
    return [getfeatures(dat, i) for i in range(len(dat))]

def f2labels(dat):
    return [label for token, postag, label in dat]

def f2tokens(dat):
    return [token for token, postag, label in dat]

In [15]:
XTr__=[f2features(x) for x in XTr_]
YTr__=[f2labels(x) for x in XTr_]

XVal__=[f2features(x) for x in XVal_]
YVal__=[f2labels(x) for x in XVal_]

In [16]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

In [17]:
crf.fit(XTr__, YTr__)

/Users/abulitibu.tuguluke/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [18]:
labls=crf.classes_
labls.remove('O')

In [19]:
y_pred = crf.predict(XVal__)
metrics.flat_f1_score(YVal__, y_pred,average='weighted', labels=labls)

0.9991524165201194

In [20]:
print(metrics.flat_classification_report(YVal__, y_pred,digits=3))

/Users/abulitibu.tuguluke/Library/Python/3.8/lib/python/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


              precision    recall  f1-score   support

           C      1.000     0.989     0.995        94
           J      1.000     1.000     1.000    109419
           O      0.995     0.998     0.997     37966
           S      0.999     0.996     0.998     41669

    accuracy                          0.999    189148
   macro avg      0.998     0.996     0.997    189148
weighted avg      0.999     0.999     0.999    189148

